In [ ]:
#!pip install dash plotly pandas requests

In [ ]:
import requests
import zipfile
import io
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from quote_chart import create_chart_app
import pandas_ta as ta

In [ ]:
print('downloading data from binance...')
# Download the ZIP file from Binance Vision
url = 'https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1m/BTCUSDT-1m-2024-02.zip'
response = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(response.content))

# Extract the CSV file from the ZIP archive
csv_filename = z.namelist()[0]
csv_file = z.open(csv_filename)

# Read the CSV file into a DataFrame
data = pd.read_csv(csv_file, header=None, names=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

# Convert the 'timestamp' column to datetime and set it as the index
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
data.set_index('timestamp', inplace=True)

# Keep only the required columns
data = data[['open', 'high', 'low', 'close', 'volume']]

print('download complete')

In [ ]:
candles_df = data.copy()

def on_period_change(button_id):
    global candles_df, selected_period
    if button_id == '':
        return
    selected_period = button_id
    candles_df = data.resample(selected_period).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    })
    candles_df = candles_df.dropna()
    
def create_figure(x0, x1):
    # slice dataframe so that there will be enough data to plot the chart and also have data on the left and right so 
    # that when the user starts to zoom/pan he will see data.
    if x0 is not None:
        # delta = x1 - x0
        # df = candles_df[x0-delta:x1+delta]
        df = candles_df[x0:x1]
    else:
        # by default show last 100 candles.
        df = candles_df[-100:]
    # define multiple panes. The top pane will be for the main price chart with candles. The second pane is for volumes.
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                vertical_spacing=0.01,
                row_heights=[0.8, 0.2, 0.2],
                specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]])
    # extended dataframe for technical analysis. 26 and 12 for MACD.
    ex = candles_df.iloc[max(0, candles_df.index.get_loc(df.index[0]) - 26 - 12):candles_df.index.get_loc(df.index[-1])]
    ema21 = ta.ema(ex['close'], 21)[-len(df):]
    fig.add_trace(go.Scatter(x=df.index, y=ema21, mode='lines', line=dict(color='blue'), name='EMA21'), row=1, col=1)
    empty_df = pd.DataFrame(columns=[None, None, None])
    macd = ta.macd(ex['close'], fast=12, slow=26, signal=9, min_periods=None, append=True)[-len(df):] if len(df) > 38 else empty_df
    fig.add_trace(go.Scatter(x=df.index, y=macd[macd.columns[0]], mode='lines', line=dict(color='blue'), name='MACD signal'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=macd[macd.columns[2]], mode='lines', line=dict(color='red'), name='MACD line'), row=3, col=1)
    fig.add_trace(go.Bar(x=df.index, y=macd[macd.columns[1]], name='MACD histogram'), row=3, col=1)
    # plot the main chart with price candles.
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['open'],
                                 high=df['high'],
                                 low=df['low'],
                                 close=df['close'],
                                 name='Prices'), row=1, col=1)
    fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume', marker=dict(color='orange')), row=2, col=1)
    # set the default dragmode to pan, remove the range slider because i use zoom/pan instead of it.
    fig.update_layout(
        dragmode='pan',
        xaxis_rangeslider_visible=False,
        width=1200, # px
        height=600,
        margin=dict(l=3, r=3, t=3, b=3),
        yaxis=dict(side='right'),
        yaxis3=dict(side='right'),
        yaxis5=dict(side='right'),
        )
    fig.update_xaxes(
        ticklabelposition="outside right",  # keep labels on the right so that they don't affect margin-left.
    )
    return fig

app = create_chart_app(create_figure, on_period_change)
app.run_server(debug=True)
